In [1]:
# https://www.kaggle.com/nicapotato/alexey-s-lightgbm-nick-mod-0-9681/code

In [2]:
import pandas as pd
import numpy as np
import time
import gc

In [3]:
path = '/home/kai/data/kaggle/talkingdata/data/'
path_train = path + 'train.csv'
path_test = path + 'test.csv'

train_cols = ['ip', 'app', 'device', 'os', 'channel', 'click_time', 'is_attributed']
test_cols = ['ip', 'app', 'device', 'os', 'channel', 'click_time']

dtypes = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'is_attributed' : 'uint8',
        'click_id'      : 'uint32'
        }
        
skip = range(1, 140000000)
print("Loading Data")
train = pd.read_csv(path_train, skiprows=skip, dtype=dtypes,
        header=0,usecols=train_cols,parse_dates=["click_time"])#.sample(1000)
test = pd.read_csv(path_test, dtype=dtypes, header=0,
        usecols=test_cols,parse_dates=["click_time"])#.sample(1000)

len_train = len(train)
print('The initial size of the train set is', len_train)
print('Binding the training and test set together...')
train=train.append(test)

del test
gc.collect()

print("Creating new time features: 'hour' and 'day'...")
train['hour'] = train["click_time"].dt.hour.astype('uint8')
train['day'] = train["click_time"].dt.day.astype('uint8')

gc.collect()

print("Creating new count features: 'n_channels', 'ip_app_count', 'ip_app_os_count'...")

print('Computing the number of channels associated with ')




Loading Data
The initial size of the train set is 44903891
Binding the training and test set together...
Creating new time features: 'hour' and 'day'...
Creating new count features: 'n_channels', 'ip_app_count', 'ip_app_os_count'...
Computing the number of channels associated with 


In [9]:
# Count by IP,DAY,HOUR
print('a given IP address within each hour...')
n_chans = train[['ip','day','hour','channel']].groupby(by=['ip','day',
          'hour'])[['channel']].count().reset_index().rename(columns={'channel': 'ip_day_hour'})


a given IP address within each hour...


In [6]:
train = train.merge(n_chans, on=['ip','day','hour'], how='left')
del n_chans
gc.collect()

183

In [7]:
train.head(30)

,app,channel,click_time,device,ip,is_attributed,os,hour,day,ip_day_hour
0,1,153,2017-11-09 02:33:33,1,32511,0.0,6,2,9,76
1,12,328,2017-11-09 02:33:33,1,154267,0.0,19,2,9,42
2,3,280,2017-11-09 02:33:33,1,34897,0.0,13,2,9,140
3,14,480,2017-11-09 02:33:33,1,191691,0.0,20,2,9,126
4,15,278,2017-11-09 02:33:33,1,56105,0.0,19,2,9,72
5,2,435,2017-11-09 02:33:33,1,191691,0.0,20,2,9,126
6,3,19,2017-11-09 02:33:33,1,210218,0.0,22,2,9,100
7,26,266,2017-11-09 02:33:33,1,96903,0.0,19,2,9,106
8,3,280,2017-11-09 02:33:33,1,44139,0.0,10,2,9,167
9,12,265,2017-11-09 02:33:33,1,162293,0.0,17,2,9,40


In [10]:
n_chans[(n_chans['ip'] == 32511) & (n_chans['day'] == 9) & (n_chans['hour'] == 2)]

,ip,day,hour,ip_day_hour
174621,32511,9,2,76
